# Case study of uniaxial compression tests

This is an example of how to use Paramaterial to process a dataset of uniaxial compression test measurements.
The data should be formatted as a set of csv files containing measurements for each test, and a single excel spreadsheet containing the metadata for the dataset.
There should be a single row in the excel spreadsheet for each csv file.

The analysis takes 4 stages:
- Data preparation
- Data processing
- Model fitting
- Test report generation

In [ ]:
import numpy as np
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd


In [ ]:
import paramaterial
import importlib
importlib.reload(paramaterial)
pam = paramaterial
DataSet, DataItem = pam.plug.DataSet, pam.plug.DataItem

## Data preparation


### Backup data and raw data
1. Create two directories: "data", "info".
2. Save all csv files into a directory called "data/00 backup data".
3. Generate a corresponding info table "info/00 backup info.xlsx" using make_info_table function. The function writes the table to a spreadsheet and returns it as a dataframe.
5. Fill in the info and save backup info and data.
6. Copy to raw info and data.

In [ ]:
backup_info = pam.preparing.make_info_table('data/00 backup data', columns=['temperature', 'rate', 'material'])
backup_info.to_excel('info/00 backup info unfilled.xlsx')
backup_info.head()

Add the metadata into the excel spreadsheet.
Apply a naming convention in the "test id" column.

In [ ]:
backup_info = pd.read_excel('info/00 backup info.xlsx')
backup_info.head()

Copy backup data and info into raw data and info, then leave backup alone in all further steps.

In [ ]:
pam.preparing.copy_data_and_info(old_data_dir='data/00 backup data', new_data_dir='data/01 prepared data',
                                 old_info_path='info/00 backup info.xlsx', new_info_path='info/01 prepared info.xlsx')

### Prepared data

Rename files according to test id.

In [ ]:
# pam.preparing.rename_by_test_id('data/01 prepared data', 'info/01 prepared info.xlsx')

Check that column headers of data files are all the same. If not the same, fix.

In [ ]:
pam.preparing.check_column_headers('data/01 prepared data')

In [ ]:
raw_dataset = DataSet('data/01 prepared data', 'info/01 prepared info.xlsx')

### Dataitem EDA
Look at one dataitem. Notice the use of the square brackets to get a subset, then to get a slice, then to get a single item.

In [ ]:
dataitem = raw_dataset[{'rate': [1], 'material': ['H560']}][0:2][0]
# dataitem = raw_dataset[{'rate': [1], 'material': ['H560']}][0:2][1]
# dataitem = raw_dataset[{'rate': [1], 'material': ['H560']}][0:2][2]

dataitem

In [ ]:
dataitem.data = dataitem.data.drop(columns=['PowAngle(deg)', 'Power(W)', 'Pram', 'Stroke(mm)', 'wedge(mm)', 'PTemp'])
sns.pairplot(pd.concat([dataitem.data.iloc[:-199:200], dataitem.data.iloc[-10:]]), hue='TC1(C)', palette='plasma')

### Dataset EDA

In [ ]:
raw_info = raw_dataset.info_table
raw_info.head()

In [ ]:
raw_info.nunique()

In [ ]:
sns.pairplot(raw_info, vars=['temperature', 'rate'], hue='material', diag_kind='hist', markers=['D', 's', 'o'],
             palette='magma')

In [ ]:
data_matrix = raw_info[['material', 'temperature', 'rate']].value_counts().sort_index()
data_matrix

Make the experimental matrix.

In [ ]:
data_matrix = data_matrix.to_frame().reset_index()
data_matrix.columns = ['material', 'temperature', 'rate', 'count']
data_matrix.pivot_table(index=['material', 'rate'], columns='temperature', values='count').fillna(0).astype(int)

Plot the dataset.

In [ ]:
# configure dataset plot
ds_plot = lambda dataset, **kwargs: pam.plotting.dataset_plot(dataset, x='Strain', y='Stress(MPa)', ylabel='Stress (MPa)',
                                                    cbar_by='temperature', cbar_label='Temperature ($^{\circ}$C)',
                                                    xlim=(-0.2, 1.5), grid=True, **kwargs)
ds_plot(raw_dataset)

In [ ]:
# configure dataset subplot
# ds_subplot = lambda dataset: pam.plotting.dataset_subplots(
#     dataset, x='Strain', y='Stress(MPa)', ylabel='Stress (MPa)',
#     nrows=3, ncols=4,
#     rows_by='material', cols_by='rate',
#     row_keys=[['AC'], ['H560'], ['H580']],
#     col_keys=[[1], [10], [50], [100]],
#     row_titles=['AC', 'H560', 'H580'],
#     col_titles=['1 s$^{-1}$', '10 s$^{-1}$', '50 s$^{-1}$', '100 s$^{-1}$'],
#     cbar_by='temperature', cbar_label='Temperature ($^{\circ}$C)',
#     xlim=(-0.2, 1.5), grid=True, wspace=0.05, hspace=0.05
# )

# ds_subplot(raw_dataset)

In [ ]:
# dataset subplots splitting low and high rates
ds_subplot = lambda dataset: pam.plotting.dataset_subplots(
    dataset, x='Strain', y='Stress(MPa)', ylabel='Stress (MPa)',
    nrows=1, ncols=2,
    rows_by='material', cols_by='rate',
    row_keys=[['AC', 'H560', 'H580']],
    col_keys=[[1, 10], [50 ,100]],
    col_titles=['1 and 10 s$^{-1}$', '50 and 100 s$^{-1}$'],
    cbar_by='temperature', cbar_label='Temperature ($^{\circ}$C)',
    xlim=(-0.2, 1.5), grid=True, wspace=0.05, hspace=0.05, figsize=(9, 5)
)

ds_subplot(raw_dataset)

## Processing
We now want to:
- Trim the data
- Apply friction corrections
- Make representative curves
- Screen the data

### Trimming the data
- Remove unnecessary columns
- Trim leading and trailing data using time-step change
- Trim remaining trailing data after max force

In [ ]:
def drop_columns(di: DataItem) -> DataItem:
    di.data = di.data.drop(columns=['PowAngle(deg)', 'Power(W)', 'Pram', 'PTemp', 'Stroke(mm)', 'wedge(mm)'])
    return di

raw_dataset = raw_dataset.apply_function(drop_columns)
raw_dataset

#### Trim using sampling rate change
After calculating the time-steps, we remove data that was sampled with a larger time-step.

In [ ]:
# calculate time-step
dataitem = raw_dataset[0]
time_step = np.diff(dataitem.data['Time(sec)'])
dataitem.data['time-step(s)'] = np.hstack([time_step[0], time_step])
ax = dataitem.data.plot(y='time-step(s)')
# trim by time-step
dataitem.data = dataitem.data[dataitem.data['time-step(s)'] < 0.02]
dataitem.data.plot(ax=ax, y='time-step(s)', label='trimmed time-step')

Trim dataset using time-step change.

In [ ]:
def trim_using_time_step_change(di: DataItem) -> DataItem:
    t_diff = np.diff(di.data['Time(sec)'])
    di.data['time diff'] = np.hstack([t_diff[0], t_diff])
    di.data = di.data[di.data['time diff'] < 0.02]
    return di

trimmed_dataset = raw_dataset.apply_function(trim_using_time_step_change)
ds_subplot(trimmed_dataset)

#### Trim using max force
The end of a compression test can be identified my the maximum force.

In [ ]:
# investigate using max force index for trimming stress
dataitem = trimmed_dataset[0]

max_force_index = dataitem.data['Force(kN)'].idxmax()
trailing_data = dataitem.data.iloc[max_force_index:].copy()
trailing_data[['trailing_stress', 'trailing_force']] = trailing_data[['Stress(MPa)', 'Force(kN)']]
trailing_data['trailing_force'] = trailing_data['Force(kN)']

ax = dataitem.data.plot(x='Strain', y=['Stress(MPa)', 'Force(kN)'])
trailing_data.plot(ax=ax, x='Strain', y=['trailing_stress', 'trailing_force'])

In [ ]:
# demonstrate max force trimming for one dataitem
def remove_trailing_data(di: DataItem):
    di.data = di.data.iloc[:di.data['Force(kN)'].idxmax()]
    return di

trimmed_dataitem = remove_trailing_data(dataitem)
trimmed_dataitem.data.plot(x='Strain', y=['Stress(MPa)', 'Force(kN)'])

In [ ]:
# apply trimming function to dataset
trimmed_dataset = trimmed_dataset.apply_function(remove_trailing_data)
ds_subplot(trimmed_dataset)

### Applying the friction correction.
$$
\sigma = P \left[\frac{h}{\mu d} \right]^{-2}\left[\exp\left(\frac{\mu d}{h}\right) - \frac{\mu d}{h} - 1 \right]^{-1}
$$

In [ ]:
trimmed_dataset[0]

In [ ]:
# demonstrate friction correction for one dataitem
dataitem = trimmed_dataset[0]

h_0 = dataitem.info['L_0'] # initial height in axial direction
d_0 = dataitem.info['D_0']  # initial diameter

h = h_0 - dataitem.data['Jaw(mm)']  # instantaneous height
d = d_0*np.sqrt(h_0/h)  # instantaneous diameter
P = dataitem.data['Force(kN)']*1000*4/(np.pi*d ** 2)  # pressure (MPa)
mu = 0.3  # friction coefficient

dataitem.data['Corrected Stress(MPa)']  = P/(1 + (mu*d)/(3*h))
dataitem.data.plot(x='Strain', y=['Stress(MPa)', 'Corrected Stress(MPa)'])

In [ ]:
# apply friction correction function to dataset
def friction_correction(di: DataItem) -> DataItem:
    mu = 0.3  # friction coefficient
    h_0 = di.info['L_0'] # initial height in axial direction
    d_0 = di.info['D_0']  # initial diameter
    h = h_0 - di.data['Jaw(mm)']  # instantaneous height
    d = d_0*np.sqrt(h_0/h)  # instantaneous diameter
    P = di.data['Force(kN)']*1000*4/(np.pi*d ** 2)  # pressure (MPa)
    di.data['Pressure(MPa)'] = P
    di.data['Stress(MPa)']  = P/(1 + (mu*d)/(3*h))  # correct stress
    return di

corrected_dataset = trimmed_dataset.apply(friction_correction)
ds_subplot(corrected_dataset)

In [ ]:
# compare corrected to un-corrected for a subset
ax = ds_plot(trimmed_dataset[{'material':['AC'], 'rate':[1]}], alpha=0.3)
ds_plot(corrected_dataset[{'material':['AC'], 'rate':[1]}], ax=ax)

### Writing processed data to new folder

In [ ]:
corrected_dataset.write_output('data/02 processed data', 'info/02 processed info.xlsx')

### Making representative curves


## Screening

## Modelling
1. Identify points of interest algorithmically
2. Fit material model